ChemBERT를 backbone으로 사용 https://huggingface.co/models?sort=downloads&search=chemBERT  
아이디어: GNN대신 트랜스포머계열을 활용해서 graph representation을 획득하는 (매우단순한) 모델  
https://dacon.io/competitions/official/236127/codeshare/8812?page=1&dtype=recent

In [2]:
# molecule predictor 
import torch
import torch.nn as nn 
import torch.nn.functional as F

from transformers import AutoModelForSequenceClassification

* Chemical data 획득

1. atomic features
- `get_atomic_features` 함수를 통해서 분자의 atomic features를 획들할 수 있음
- features는 `mendeleev` 패키지에서 획득했고, `Chemical_feature_generator.mendeleev_atomic_f`를 확인하면 어떠한 atomic feature를 선택했는지 확인할 수 있음
- `generate_mol_atomic_features` 함수를 통해서 하나의 분자에 해당하는 atomic feature matrix를 획득할 수 있음
- `get_adj_matrix` 함수를 통해서 분자의 adjacency matrix를 획득하고, 이를 다시 sparse tensor의 형태로 변형함
- atomic feature와 함께 사용하면 GCN의 forward 함수에 활용할 수 있음
    - *deepchem MPNN featurizer가 제공하는 것과 많은 부분이 유사
- `get_atomic_feature_from_deepchem`을 통해서 더 다양한 범위의 atomic feature를 획득할 수 있으나, 대회에서 제공해준 모든 데이터에 유효한 동작을 하는진 확인 필요
- `encoder_smiles` 함수는 chemBERT의 입력을 위해 smiles tokenizing을 하는 함수임

2. molecular features
- `get_molecular_features` 함수를 통해서 분자의 molecular property를 획득할 수 있고 `rdKit`을 이용함
    - 이렇게 획드간 feature와 대회에서 제공된 동일한 특성 값 사이에 차이가 있음 

In [ ]:
C